In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('roc').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
df = spark.sql('select* from movielens_ratings')

In [4]:
df.show(5)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
+-------+------+------+
only showing top 5 rows

In [5]:
df.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [6]:
train,test = df.randomSplit([0.8,0.2])

In [7]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [8]:
model = als.fit(train)

In [9]:
# predicts the rating
predictions = model.transform(test)

In [10]:
predictions.show(5)

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 4.0| 12| 0.5105163|
 31| 1.0| 24|0.23853058|
 85| 1.0| 15| 1.19667|
 85| 1.0| 2| 2.8669956|
 53| 3.0| 13| 2.428243|
+-------+------+------+----------+
only showing top 5 rows

In [11]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [12]:
rmse = evaluator.evaluate(predictions)

In [13]:
print('RMSE')
print(rmse)

RMSE
1.79070573024

In [14]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 9| 11|
 21| 11|
 22| 11|
 23| 11|
 25| 11|
 32| 11|
 61| 11|
 62| 11|
 66| 11|
 67| 11|
 70| 11|
 72| 11|
 76| 11|
 78| 11|
 82| 11|
 97| 11|
+-------+------+

In [15]:
recommendations = model.transform(single_user)

In [16]:
recommendations.orderBy('Prediction',ascending = False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 23| 11| 3.282947|
 32| 11| 3.1446016|
 72| 11| 2.5078497|
 82| 11| 1.7269955|
 78| 11| 0.8727428|
 76| 11| 0.7903875|
 21| 11| 0.6787041|
 66| 11| 0.65829587|
 70| 11| 0.46365088|
 22| 11| 0.25124627|
 9| 11|0.076968625|
 97| 11|-0.12006426|
 67| 11|-0.16227065|
 61| 11|-0.55970496|
 25| 11| -0.6497287|
 62| 11| -1.6814041|
+-------+------+-----------+